In [51]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
from tensorflow.keras.layers import Normalization, Dense, InputLayer
from tensorflow.keras.losses import MeanSquaredError, Huber, MeanAbsoluteError
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,OrdinalEncoder, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('D:/04_Lesson_IT/12_Data_Sicense/03_Machine_Learing/Project_ML/Dataset/diabetes_binary_5050split_health_indicators_BRFSS2015.csv')

In [27]:
df.head(2)

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70692 entries, 0 to 70691
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Diabetes_binary       70692 non-null  float64
 1   HighBP                70692 non-null  float64
 2   HighChol              70692 non-null  float64
 3   CholCheck             70692 non-null  float64
 4   BMI                   70692 non-null  float64
 5   Smoker                70692 non-null  float64
 6   Stroke                70692 non-null  float64
 7   HeartDiseaseorAttack  70692 non-null  float64
 8   PhysActivity          70692 non-null  float64
 9   Fruits                70692 non-null  float64
 10  Veggies               70692 non-null  float64
 11  HvyAlcoholConsump     70692 non-null  float64
 12  AnyHealthcare         70692 non-null  float64
 13  NoDocbcCost           70692 non-null  float64
 14  GenHlth               70692 non-null  float64
 15  MentHlth           

In [21]:
df.columns

Index(['Diabetes_binary', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
       'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')

In [29]:
df['Diabetes_binary'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 70692 entries, 0 to 70691
Series name: Diabetes_binary
Non-Null Count  Dtype  
--------------  -----  
70692 non-null  float64
dtypes: float64(1)
memory usage: 552.4 KB


In [33]:
df['Diabetes_binary'].value_counts()

Diabetes_binary
0.0    35346
1.0    35346
Name: count, dtype: int64

In [35]:
df['Diabetes_binary'] = df['Diabetes_binary'].replace({'1.0':1,'0.0':0})

In [39]:
df.nunique()

Diabetes_binary          2
HighBP                   2
HighChol                 2
CholCheck                2
BMI                     80
Smoker                   2
Stroke                   2
HeartDiseaseorAttack     2
PhysActivity             2
Fruits                   2
Veggies                  2
HvyAlcoholConsump        2
AnyHealthcare            2
NoDocbcCost              2
GenHlth                  5
MentHlth                31
PhysHlth                31
DiffWalk                 2
Sex                      2
Age                     13
Education                6
Income                   8
dtype: int64

In [41]:
numerical = []
categorical = []
# excluded_columns  = ['HighBP','HighChol','CholCheck']
excluded_columns = []
target = 'Diabetes_binary'
for column in df.columns:
    if column == target:
        continue
    elif column in excluded_columns:
        continue
    elif df[column].nunique()==2:
        categorical.append(column)
    elif df[column].dtype != 'O':
        numerical.append(column)

In [43]:
numerical

['BMI', 'GenHlth', 'MentHlth', 'PhysHlth', 'Age', 'Education', 'Income']

In [47]:
df[categorical]

,HighBP,HighChol,CholCheck,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,DiffWalk,Sex
0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
3,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70687,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
70688,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
70689,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
70690,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [53]:
OrdinalEncoder().fit_transform(df[categorical])

array([[1., 0., 1., ..., 0., 0., 1.],
       [1., 1., 1., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [1., 1., 1., ..., 0., 1., 0.],
       [1., 1., 1., ..., 0., 1., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [55]:
numerical_transformer = Pipeline([
    ('imput', SimpleImputer(strategy='median')), ## mean , median: Number, 
    ('scaling', StandardScaler())
])
categorical_transformer = Pipeline([
    ('inmute', SimpleImputer(strategy='most_frequent')), ## Categorical 
     ('encoding', OneHotEncoder())
])
preprocessor = ColumnTransformer([
    ('numerical', numerical_transformer, numerical),
    ('categorical', categorical_transformer, categorical)
])  ### 

In [57]:
preprocessor.fit_transform(df)

array([[-0.54217564,  0.1463042 ,  0.15301974, ...,  0.        ,
         0.        ,  1.        ],
       [-0.54217564,  0.1463042 , -0.46005828, ...,  0.        ,
         0.        ,  1.        ],
       [-0.54217564, -1.64974275, -0.46005828, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.68274544,  1.94235114,  1.37917577, ...,  1.        ,
         1.        ,  0.        ],
       [-1.66673405,  1.04432767, -0.46005828, ...,  1.        ,
         1.        ,  0.        ],
       [-0.68274544, -0.75171927, -0.46005828, ...,  0.        ,
         1.        ,  0.        ]])

In [59]:
import joblib

In [61]:
joblib.dump(preprocessor, 'preprocessor.pkl')

['preprocessor.pkl']

In [65]:
x = df.drop(columns=target)
y = df[target]
X_transformed = preprocessor.fit_transform(x)


In [67]:
X_transformed

array([[-0.54217564,  0.1463042 ,  0.15301974, ...,  0.        ,
         0.        ,  1.        ],
       [-0.54217564,  0.1463042 , -0.46005828, ...,  0.        ,
         0.        ,  1.        ],
       [-0.54217564, -1.64974275, -0.46005828, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.68274544,  1.94235114,  1.37917577, ...,  1.        ,
         1.        ,  0.        ],
       [-1.66673405,  1.04432767, -0.46005828, ...,  1.        ,
         1.        ,  0.        ],
       [-0.68274544, -0.75171927, -0.46005828, ...,  0.        ,
         1.        ,  0.        ]])

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)
input_shape = X_train.shape[1]

In [75]:
  # Determine input shape dynamically
model = tf.keras.Sequential([
    InputLayer(input_shape=(input_shape,)),
    Dense(256, activation="relu"), 
    Dense(128, activation="relu"),
    Dense(64, activation="relu"),
    Dense(32, activation="relu"),
    Dense(8, activation="relu"),
    Dense(2, activation='sigmoid') ### 
])

In [77]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
1768/1768 ━━━━━━━━━━━━━━━━━━━━ 22s 9ms/step - accuracy: 0.7404 - loss: 0.5252 - val_accuracy: 0.7521 - val_loss: 0.5058
Epoch 2/10
1768/1768 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.7514 - loss: 0.5051 - val_accuracy: 0.7551 - val_loss: 0.5021
Epoch 3/10
1768/1768 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.7539 - loss: 0.4993 - val_accuracy: 0.7542 - val_loss: 0.5038
Epoch 4/10
1768/1768 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.7529 - loss: 0.5039 - val_accuracy: 0.7513 - val_loss: 0.5047
Epoch 5/10
1768/1768 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.7528 - loss: 0.5002 - val_accuracy: 0.7534 - val_loss: 0.5010
Epoch 6/10
1768/1768 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.7599 - loss: 0.5001 - val_accuracy: 0.7538 - val_loss: 0.5013
Epoch 7/10
1768/1768 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.7549 - loss: 0.4993 - val_accuracy: 0.7527 - val_loss: 0.5049
Epoch 8/10
1768/1768 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.7574 - loss: 0

In [79]:
model.save("model.h5")